### For this second part of the challenge we gonna simulate that we have a database with the tables provided and we gonna use SQL to query the data and get the results we need.

### Preparing the data 

In [16]:
import sqlite3
import pandas as pd

vuelos = pd.DataFrame(data=[(1, 1, 1, '2021-05-02'),
                (2, 1, 1, '2021-05-02'),
                (3, 2, 2, '2021-05-02'),
                (4, 3, 2, '2021-05-02'),
                (1, 3, 2, '2021-05-02'),
                (2, 1, 1, '2021-05-02'),
                (2, 3, 1, '2021-05-04'),
                (3, 4, 1, '2021-05-04'),
                (3, 4, 1, '2021-05-04')], columns=["id_aerolinea", "id_aeropuerto", "id_movimiento", "dia"])

aerolineas = pd.DataFrame(data=[
                 (1, 'Volaris'),
                 (2, 'Aeromar'),
                 (3, 'Interjet'),
                 (4, 'Aeromexico')] , columns=["id_aerolinea", "nombre_aerolinea"])

aeropuertos = pd.DataFrame(data=[
                 (1, 'Benito Juarez'),
                 (2, 'Guanajuato'),
                 (3, 'La paz'),
                 (4, 'Oaxaca')], columns=["id_aeropuerto", "nombre_aeropuerto"])

movimientos = pd.DataFrame(data=[
                    (1, 'Salida'),
                    (2, 'Llegada')] , columns=["id_movimiento", "descripcion"])


### Creating Database and Tables

In [17]:
# we use sqlite3 to create a database
import os
if os.path.exists("AeroViajes.db"):
    os.remove("AeroViajes.db")

conn = sqlite3.connect('AeroViajes.db')
c = conn.cursor()

In [18]:

# we create the tables
c.execute('''CREATE TABLE aerolineas
                (id_aerolinea INTEGER PRIMARY KEY,
                 nombre_aerolinea VARCHAR(50))''')

c.execute('''CREATE TABLE aeropuertos 
                (id_aeropuerto INTEGER PRIMARY KEY,
                 nombre_aeropuerto  VARCHAR(50))''')

c.execute('''CREATE TABLE movimientos 
                (id_movimiento INTEGER PRIMARY KEY,
                 descripcion VARCHAR(25))''')

c.execute('''CREATE TABLE vuelos(
                 id_aerolinea INTEGER NOT NULL,
                 id_aeropuerto INTEGER NOT NULL,
                 id_movimiento INTEGER NOT NULL,
                 dia DATETIME,
                 FOREIGN KEY (id_aerolinea) REFERENCES aerolineas(id_aerolinea),
                 FOREIGN KEY (id_aeropuerto) REFERENCES aeropuertos(id_aeropuerto),
                 FOREIGN KEY (id_movimiento) REFERENCES movimientos(id_movimiento))''')


conn.commit()

c.close()

In [19]:
# we test if the tables were created, showing all tables available
c = conn.cursor()

c.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(c.fetchall())

c.close()


[('aerolineas',), ('aeropuertos',), ('movimientos',), ('vuelos',)]


In [20]:
c = conn.cursor()
c.execute("SELECT * FROM aerolineas")
print(c.fetchall())

c.close()

[]


### loading the data into the tables 

In [21]:
# next step is to load the database filling all the tables with the data of the challenge
c = conn.cursor()


def load_data(table_name, df):
    for index, row in df.iterrows():

        query = f"INSERT INTO {table_name} VALUES ({', '.join(['?'] * len(row))})"
        c.execute(query, tuple(row))


# table viajes
load_data("vuelos", vuelos)
# table aerolineas
load_data("aerolineas", aerolineas)
# table aeropuertos
load_data("aeropuertos", aeropuertos)
# table movimientos
load_data("movimientos", movimientos)

conn.commit()
c.close()

In [22]:
c = conn.cursor()

c.execute("""SELECT * FROM vuelos
          """)
dia = c.fetchall()

print(dia)



[(1, 1, 1, '2021-05-02'), (2, 1, 1, '2021-05-02'), (3, 2, 2, '2021-05-02'), (4, 3, 2, '2021-05-02'), (1, 3, 2, '2021-05-02'), (2, 1, 1, '2021-05-02'), (2, 3, 1, '2021-05-04'), (3, 4, 1, '2021-05-04'), (3, 4, 1, '2021-05-04')]


### ¿Cuál es el nombre aeropuerto que ha tenido mayor movimiento durante el año?

In [23]:
c = conn.cursor()
c.execute('''SELECT ap.nombre_aeropuerto, count(v.id_movimiento) as total_movimientos
             FROM aeropuertos as ap
             LEFT JOIN vuelos as v 
             ON ap.id_aeropuerto = v.id_aeropuerto
             GROUP BY ap.nombre_aeropuerto
             ORDER BY total_movimientos DESC
          ''')
aeropuerto = c.fetchall()
conn.commit()
c.close()
print(aeropuerto)

[('La paz', 3), ('Benito Juarez', 3), ('Oaxaca', 2), ('Guanajuato', 1)]


In [24]:
# De acuerdo a los datos,la paz y benito juarez son los aeropuertos con mayor numero de movimientos o vuelos
c = conn.cursor()
query = '''SELECT ap.nombre_aeropuerto, count(v.id_movimiento) as total_movimientos
             FROM aeropuertos as ap
             LEFT JOIN vuelos as v 
             ON ap.id_aeropuerto = v.id_aeropuerto
             GROUP BY ap.nombre_aeropuerto
             ORDER BY total_movimientos DESC
          '''

aeropuerto = pd.read_sql_query(query, conn)
print(aeropuerto)

  nombre_aeropuerto  total_movimientos
0            La paz                  3
1     Benito Juarez                  3
2            Oaxaca                  2
3        Guanajuato                  1


### ¿Cuál es el nombre aerolínea que ha realizado mayor número de vuelos durante el año?

In [25]:

c = conn.cursor()
c.execute('''SELECT ae.nombre_aerolinea, count(v.id_movimiento) as total_vuelos
             FROM aerolineas as ae
             INNER JOIN vuelos as v ON 
             ae.id_aerolinea = v.id_aerolinea
             GROUP BY ae.nombre_aerolinea
             ORDER BY total_vuelos DESC
            ''')
aerolinea = c.fetchall()
conn.commit()
c.close()

print(aerolinea)


[('Interjet', 3), ('Aeromar', 3), ('Volaris', 2), ('Aeromexico', 1)]


In [26]:
# De acuerdo a los datos interjet y aeromar son las aerolineas con mayor numero de vuelos
df = pd.DataFrame(aerolinea, columns = ['aerolinea', 'total_movimientos'])
df


,aerolinea,total_movimientos
0,Interjet,3
1,Aeromar,3
2,Volaris,2
3,Aeromexico,1


### ¿En qué día se han tenido mayor número de vuelos?

In [27]:
c = conn.cursor()
c.execute(""" SELECT v.dia, COUNT(v.id_movimiento) as total_vuelos
                FROM vuelos as v
                GROUP BY v.dia
                ORDER BY total_vuelos DESC
            """)

dia = c.fetchall()
conn.commit()
c.close()
dia

[('2021-05-02', 6), ('2021-05-04', 3)]

In [28]:
# Los el dia con mayor numero de vuelos es el 2021-05-02
df = pd.DataFrame(dia, columns = ['dia', 'total_salidas'])
df

,dia,total_salidas
0,2021-05-02,6
1,2021-05-04,3


### ¿Cuáles son las aerolíneas que tienen mas de 2 vuelos por día?

In [29]:
c = conn.cursor()
query = """ SELECT ae.nombre_aerolinea, count(ae.nombre_aerolinea) as total_vuelos
                FROM aerolineas as ae
                INNER JOIN vuelos as v
                ON ae.id_aerolinea = v.id_aerolinea
                GROUP BY v.dia, ae.nombre_aerolinea
                HAVING COUNT(v.dia) > 2
                ORDER BY total_vuelos DESC
            """
c.execute(query)
aerolinea = c.fetchall()
conn.commit()
c.close()

aerolinea

[]

In [30]:
#no existe aerolinea con mas de 2 vuelos en un dia
aerolinea = pd.read_sql_query(query, conn)
aerolinea

,nombre_aerolinea,total_vuelos
